In [1]:
import os
from pathlib import Path
from aocd.models import Puzzle
import collections
import numpy as np
import re
from statistics import median, mean
import math
import queue
import itertools
import more_itertools
infinite_defaultdict = lambda: collections.defaultdict(infinite_defaultdict)
from copy import copy
import functools
import heapq
import operator
from tqdm.notebook import trange, tqdm
import networkx as nx
import pprint
from PIL import Image, ImageDraw


# Day 1

In [2]:
puzzle = Puzzle(2024, 1)

In [3]:
data = [list(map(int, line.split("   "))) for line in puzzle.input_data.split("\n")]

## Part 1 

In [4]:
sorted_data = [sorted(ls) for ls in zip(*data)]
res_a = sum(abs(x - y) for x, y in zip(*sorted_data))

In [5]:
puzzle.answer_a = res_a

## Part 2

In [6]:
x_counted = collections.Counter(x for x, _ in data)
y_counted = collections.Counter(y for _, y in data)

In [7]:
res_b = sum(xk * xv * y_counted[xk] for xk, xv in x_counted.items())

In [8]:
puzzle.answer_b = res_b

# Day 2

In [9]:
puzzle = Puzzle(2024, 2)

In [10]:
data = [list(map(int, line.split(" "))) for line in puzzle.input_data.split("\n")]

In [11]:
data_test = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9"""

In [12]:
data_test = [list(map(int, line.split(" "))) for line in data_test.split("\n")]

In [13]:
data_test

[[7, 6, 4, 2, 1],
 [1, 2, 7, 8, 9],
 [9, 7, 6, 2, 1],
 [1, 3, 2, 4, 5],
 [8, 6, 4, 4, 1],
 [1, 3, 6, 7, 9]]

## Part 1 

In [14]:
def valid_ascend(seq):
    return all(a < b < a + 4 for a, b in itertools.pairwise(seq))

def valid_descend(seq):
    return all(a > b > a - 4 for a, b in itertools.pairwise(seq))
    
def valid_sequence(seq):
    return valid_ascend(seq) or valid_descend(seq)


In [15]:
res_a = sum(valid_sequence(sequence) for  sequence in data)
res_a

564

In [16]:
puzzle.answer_a = res_a

## Part 2

In [17]:
def valid_ascend_b(seq):
    for i, (a, b) in enumerate(itertools.pairwise(seq)):
        if not (a < b < a + 4):
            return False, i
    return True, len(seq)
            

def valid_descend_b(seq):
    for i, (a, b) in enumerate(itertools.pairwise(seq)):
        if not (a > b > a - 4):
            return False, i
    return True, len(seq)

def valid_sequence_b(sequence):
    valid, i = valid_ascend_b(sequence)
    if valid:
        return True

    for offset in range(2):
        valid, _ = valid_ascend_b(sequence[i - 1 + offset: i + offset] + sequence[i + 1 + offset:])
        if valid:
            return True

    valid, i = valid_descend_b(sequence)
    if valid:
        return True
        
    for offset in range(2):
        valid, _ = valid_descend_b(sequence[i - 1 + offset: i + offset] + sequence[i + 1 + offset:])
        if valid:
            return True
            
    return False
    

In [18]:
res_b = sum(valid_sequence_b(sequence) for sequence in data)
res_b

604

In [19]:
puzzle.answer_b = res_b

# Day 3

In [20]:
puzzle = Puzzle(2024, 3)

In [21]:
data_test = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

## Part 1 

In [22]:
mul_regex = re.compile("mul\((?P<X>\d{1,3}),(?P<Y>\d{1,3})\)")
def parse_mul(data):
    return sum(int(m["X"])*int(m["Y"]) for m in re.finditer(mul_regex, data))

In [23]:
res_a = parse_mul(puzzle.input_data)
res_a

175700056

In [24]:
puzzle.answer_a = res_a

## Part 2

In [25]:
remove_dont_regex = re.compile(r"don't\(\).*?(?=(do(n't)?\(\)|$))", re.S)

In [26]:
re.match(re.compile(".*"), "\n")

<re.Match object; span=(0, 0), match=''>

In [27]:
data_test = "don't()don't()bdon't()cdo()adon't()\ndo()ddodon't()d"

In [28]:
for m in re.finditer(remove_dont_regex, data_test):
    print(m.group())
    print()

don't()

don't()b

don't()c

don't()


don't()d



In [29]:
data_do = re.sub(remove_dont_regex, "", puzzle.input_data)

In [30]:
answer_b = parse_mul(data_do)
answer_b

71668682

In [31]:
puzzle.answer_b = answer_b

# Day 4

In [32]:
puzzle = Puzzle(2024, 4)

In [33]:
data_test = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
grid_test = np.array([list(line) for line in data_test.split("\n")])
grid = np.array([list(line) for line in puzzle.input_data.split("\n")])

## Part 1 

In [34]:
def count_occurence(lst: list[str], word: str) -> int:
    lst_str = "".join(lst)
    # print(f"{lst_str:10} -> {word}: {lst_str.count(word)} {word[::-1]}: {lst_str.count(word[::-1])}")
    return lst_str.count(word) + lst_str.count(word[::-1])

In [35]:
def get_all_word_occurence(grid, word):
    occurences = 0
    x, y = grid.shape
    gird_sym = np.fliplr(grid)
    
    for xi in range(x):
        occurences += count_occurence(grid[xi, :], word)
    
    for yi in range(y):
        occurences += count_occurence(grid[:, yi], word)
    
    for di in range(-x + 1, y):
        occurences += count_occurence(grid.diagonal(di), word)
        occurences += count_occurence(gird_sym.diagonal(di), word)

    return occurences
        

In [36]:
get_all_word_occurence(grid_test, word)

NameError: name 'word' is not defined

In [ ]:
answer_a = get_all_word_occurence(grid, word)
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
def get_x_mas_occurences(grid):
    occurences = 0
    x, y = grid.shape
    for xi in range(1, x - 1):
        for yi in range(1, y - 1):
            if grid[xi, yi] != "A":
                continue
            corner = f"{grid[xi-1, yi-1]}{grid[xi-1, yi+1]}{grid[xi+1, yi+1]}{grid[xi+1, yi-1]}"
            occurences += corner in ["SSMM", "SMMS", "MMSS", "MSSM"]

    return occurences

In [ ]:
get_x_mas_occurences(grid_test)

In [ ]:
answer_b = get_x_mas_occurences(grid)
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 5

In [38]:
puzzle = Puzzle(2024, 5)

In [39]:
constraints, data = puzzle.input_data.split("\n\n")

In [40]:
input_data_test = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
"""

## Part 1 

In [65]:
def parse_data_a(data_input):
    constraints, updates = data_input.split("\n\n")
    
    before_constraints = collections.defaultdict(set)
    
    for constraint in constraints.split():
        X, Y = map(int, constraint.split("|"))
        before_constraints[Y].add(X)

    counter_a = 0
    for update in updates.split():
        update_pages = list(map(int, update.split(",")))
        invalid_pages = set()
        for i, page in enumerate(update_pages):
            if page in invalid_pages:
                break
                
            invalid_pages.update(before_constraints[page])
        else:
            mid_element = update_pages[(len(update_pages) - 1)//2]
            counter_a += mid_element

    return counter_a

In [63]:
parse_data_a(input_data_test)

143

In [64]:
answer_a = parse_data_a(puzzle.input_data)
answer_a

4135

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [70]:
def parse_data_b(data_input):
    constraints, updates = data_input.split("\n\n")
    
    before_constraints = collections.defaultdict(set)
    after_constraints = collections.defaultdict(set)
    
    for constraint in constraints.split():
        X, Y = map(int, constraint.split("|"))
        before_constraints[Y].add(X)
        after_constraints[X].add(Y)

    invalid_updates = []
    for update in updates.split():
        update_pages = list(map(int, update.split(",")))
        invalid_pages = set()
        for i, page in enumerate(update_pages):
            if page in invalid_pages:
                invalid_updates.append(update_pages)
                break
                
            invalid_pages.update(before_constraints[page])
    
    counter_b = 0
    while invalid_updates:
        invalid_update = invalid_updates.pop()
        invalid_pages = set()
        for i, page in enumerate(invalid_update):
            if page in invalid_pages:
                for j, before_page in enumerate(invalid_update[:i]):
                    if before_page in after_constraints[page]:
                        new_update = invalid_update[:j] + [page] + invalid_update[j:i] + invalid_update[i+1:]
                        invalid_updates.append(new_update)
                        break
                break

            invalid_pages.update(before_constraints[page])
        else:
            mid_element = invalid_update[(len(invalid_update) - 1)//2]
            counter_b += mid_element
        
    return counter_b        

In [72]:
parse_data_b(input_data_test)

123

In [73]:
answer_b = parse_data_b(puzzle.input_data)
answer_b

5285

In [58]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 5! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 6

In [2]:
puzzle = Puzzle(2024, 6)

In [3]:
data_test = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""

In [4]:
class Patrol:
    def __init__(self, input_data, time_parardox_position=None):
        self.obstruction_symbol = "#"
        self.visited_symbol = "X"
        self.time_parardox_symbol = "O"
        self.input_data = input_data
        self.time_parardox_position = time_parardox_position    
        self.data_map = self.init_data()
        if time_parardox_position is not None:
            self.data_map[time_parardox_position] = self.obstruction_symbol
        
        self.direction_map = self.data_map.copy()
        self.directions_mapping = {
            "<": (0, -1),
            "^": (-1, 0),
            ">": (0, 1),
            "V": (1, 0)
        }
        self.directions = [*self.directions_mapping]
        self.guard_direction = None
        self.guard_position = None

        self.starting_guard_position = self.find_guard_position()
        self.valid_time_paradox = self.move_guard()
        if time_parardox_position:
            self.data_map[time_parardox_position] = self.time_parardox_symbol

    def init_data(self):
        return np.array(list(map(list, self.input_data.split("\n"))))

    def find_guard_position(self):
        for direction in self.directions_mapping:
            if coord := (np.argwhere(self.data_map == direction).tolist()):
                guard_position = tuple(coord[0])
                self.guard_position = guard_position
                self.guard_direction  = self.data_map[guard_position]
                return guard_position

    def move_guard(self):
        hold = False

        while True :
            self.data_map[self.guard_position] = self.visited_symbol
            if not hold:
                # whenever a guard hit a obstuction, we want to keep the way he hit it instead of writing his next move
                self.direction_map[self.guard_position] = self.guard_direction
            else:
                hold = False
            
            next_guard_position = self.new_position(self.guard_position, self.guard_direction)
            
            # Out of bound end part a
            if not self.position_in_map(next_guard_position):
                return False

            # Already visited position with same direction part b
            if self.direction_map[next_guard_position] == self.guard_direction:
                return True

            if self.data_map[next_guard_position] == self.obstruction_symbol:
                self.rotate_guard()
                hold = True
            else:
                self.guard_position = next_guard_position
                self.data_map[self.guard_position] = self.guard_direction

    def new_position(self, position, direction):
        return tuple(g_p + d for g_p, d in zip(position, self.directions_mapping[direction]))

    def rotate_guard(self):
        self.guard_direction = self.next_direction()
        self.data_map[self.guard_position] = self.guard_direction
    
    def next_direction(self):
        return self.directions[(self.directions.index(self.guard_direction) + 1) % len(self.directions)]

    def position_in_map(self, position):
        x, y = position
        shape_x, shape_y = self.data_map.shape
        return 0 <= x < shape_x and 0 <= y < shape_y

    def part_a(self):
        pprint.pp(self.data_map)
        return collections.Counter(self.data_map.flatten())[self.visited_symbol]
    
    def part_b(self):
        data_input = self.data_map.copy()
        valid_time_paradox = 0
        time_paradoxes = set(map(tuple, np.argwhere(self.data_map == "X").tolist()))
        
        for coord in tqdm(time_paradoxes - {self.starting_guard_position}):
            patrol = self.__class__(self.input_data, coord)
            if patrol.valid_time_paradox:
                valid_time_paradox += 1
                self.data_map[coord] = self.time_parardox_symbol

        pprint.pp(self.data_map)
        return valid_time_paradox

In [5]:
patrol_test = Patrol(data_test)

In [6]:
patrol = Patrol(puzzle.input_data)

## Part 1 

In [7]:
patrol_test.part_a()

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', 'X', 'X', 'X', 'X', 'X', '#'],
       ['.', '.', '.', '.', 'X', '.', '.', '.', 'X', '.'],
       ['.', '.', '#', '.', 'X', '.', '.', '.', 'X', '.'],
       ['.', '.', 'X', 'X', 'X', 'X', 'X', '#', 'X', '.'],
       ['.', '.', 'X', '.', 'X', '.', 'X', '.', 'X', '.'],
       ['.', '#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.'],
       ['.', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '#', '.'],
       ['#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', 'X', '.', '.']], dtype='<U1')


41

In [8]:
answer_a = patrol.part_a()
answer_a

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')


4602

In [9]:
puzzle.answer_a = answer_a

## Part 2

In [10]:
patrol_test.part_b()

  0%|          | 0/40 [00:00<?, ?it/s]

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', 'X', 'X', 'X', 'X', 'X', '#'],
       ['.', '.', '.', '.', 'X', '.', '.', '.', 'X', '.'],
       ['.', '.', '#', '.', 'X', '.', '.', '.', 'X', '.'],
       ['.', '.', 'X', 'X', 'X', 'X', 'X', '#', 'X', '.'],
       ['.', '.', 'X', '.', 'X', '.', 'X', '.', 'X', '.'],
       ['.', '#', 'X', 'O', 'X', 'X', 'X', 'X', 'X', '.'],
       ['.', 'X', 'X', 'X', 'X', 'X', 'O', 'O', '#', '.'],
       ['#', 'O', 'X', 'O', 'X', 'X', 'X', 'X', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', 'O', '.', '.']], dtype='<U1')


6

In [11]:
answer_b = patrol.part_b()
answer_b

  0%|          | 0/4601 [00:00<?, ?it/s]

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')


1703

In [12]:
puzzle.answer_b = answer_b

# Day 7

In [13]:
puzzle = Puzzle(2024, 7)

In [14]:
data = puzzle.input_data.split("\n")

In [15]:
data_test = """190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20"""

In [16]:
class TruthValue:
    def __init__(self, input_data_line, debug=False):
        self.total, self.numbers = self.parse_input(input_data_line)
        self.__valid_a = None
        self.__valid_b = None
        self.debug = debug

    def parse_input(self, input_data_line):
            total, numbers = input_data_line.split(":")
            return int(total), list(map(int, numbers.strip().split(" ")))

    @property
    def valid_a(self):
        if self.__valid_a is None:
            self.__valid_a = self._parse(self.numbers[0], self.numbers[1:], f"{self.numbers[0]}", part="a")
        
        return self.__valid_a

    @property
    def valid_b(self):
        if self.__valid_b is None:
            self.__valid_b = self._parse(self.numbers[0], self.numbers[1:], f"{self.numbers[0]}", part="b")
        
        return self.__valid_b

    def _parse(self, total, numbers, tracking, part):
        if self.debug:
            print(f"{self.total} -> {total} = {tracking}")
        if not numbers:
            return self.total == total
        # Early break
        if total > self.total:
            return False
            
        first_n = numbers.pop(0)

        if part == "b":
            valid = self._parse(int(f"{total}{first_n}"), numbers[:], f"{tracking} || {first_n}", part)
            # Early break b
            if valid:
                return valid
                
        valid = self._parse(total + first_n, numbers[:], f"{tracking} + {first_n}", part)
        # Early break 2
        if valid:
            return valid
        
        valid = self._parse(total * first_n, numbers[:], f"{tracking} * {first_n}", part)
        return valid        
        

In [17]:
tvs_test = [TruthValue(line, debug=True) for line in data_test.split("\n")]
tvs = [TruthValue(line) for line in puzzle.input_data.split("\n")]

## Part 1 

In [18]:
sum(tv_test.total for tv_test in tvs_test if tv_test.valid_a)

190 -> 10 = 10
190 -> 29 = 10 + 19
190 -> 190 = 10 * 19
3267 -> 81 = 81
3267 -> 121 = 81 + 40
3267 -> 148 = 81 + 40 + 27
3267 -> 3267 = 81 + 40 * 27
83 -> 17 = 17
83 -> 22 = 17 + 5
83 -> 85 = 17 * 5
156 -> 15 = 15
156 -> 21 = 15 + 6
156 -> 90 = 15 * 6
7290 -> 6 = 6
7290 -> 14 = 6 + 8
7290 -> 20 = 6 + 8 + 6
7290 -> 35 = 6 + 8 + 6 + 15
7290 -> 300 = 6 + 8 + 6 * 15
7290 -> 84 = 6 + 8 * 6
7290 -> 99 = 6 + 8 * 6 + 15
7290 -> 1260 = 6 + 8 * 6 * 15
7290 -> 48 = 6 * 8
7290 -> 54 = 6 * 8 + 6
7290 -> 69 = 6 * 8 + 6 + 15
7290 -> 810 = 6 * 8 + 6 * 15
7290 -> 288 = 6 * 8 * 6
7290 -> 303 = 6 * 8 * 6 + 15
7290 -> 4320 = 6 * 8 * 6 * 15
161011 -> 16 = 16
161011 -> 26 = 16 + 10
161011 -> 39 = 16 + 10 + 13
161011 -> 338 = 16 + 10 * 13
161011 -> 160 = 16 * 10
161011 -> 173 = 16 * 10 + 13
161011 -> 2080 = 16 * 10 * 13
192 -> 17 = 17
192 -> 25 = 17 + 8
192 -> 39 = 17 + 8 + 14
192 -> 350 = 17 + 8 * 14
192 -> 136 = 17 * 8
192 -> 150 = 17 * 8 + 14
192 -> 1904 = 17 * 8 * 14
21037 -> 9 = 9
21037 -> 16 = 9 + 7
21

3749

In [20]:
answer_b = 0
for tv in tqdm(tvs):
    if tv.valid_a:
        answer_a += tv.total
answer_a

  0%|          | 0/850 [00:00<?, ?it/s]

945512586797

In [21]:
puzzle.answer_a = answer_a

aocd will not submit that answer. At 2024-12-07 05:09:12.745108-05:00 you've previously submitted 945512582195 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]
It is certain that '945512586797' is incorrect, because '945512586797' != '945512582195'.


## Part 2

In [22]:
sum(tv_test.total for tv_test in tvs_test if tv_test.valid_b)

190 -> 10 = 10
190 -> 1019 = 10 || 19
190 -> 29 = 10 + 19
190 -> 190 = 10 * 19
3267 -> 81 = 81
3267 -> 8140 = 81 || 40
3267 -> 121 = 81 + 40
3267 -> 12127 = 81 + 40 || 27
3267 -> 148 = 81 + 40 + 27
3267 -> 3267 = 81 + 40 * 27
83 -> 17 = 17
83 -> 175 = 17 || 5
83 -> 22 = 17 + 5
83 -> 85 = 17 * 5
156 -> 15 = 15
156 -> 156 = 15 || 6
7290 -> 6 = 6
7290 -> 68 = 6 || 8
7290 -> 686 = 6 || 8 || 6
7290 -> 68615 = 6 || 8 || 6 || 15
7290 -> 701 = 6 || 8 || 6 + 15
7290 -> 10290 = 6 || 8 || 6 * 15
7290 -> 74 = 6 || 8 + 6
7290 -> 7415 = 6 || 8 + 6 || 15
7290 -> 89 = 6 || 8 + 6 + 15
7290 -> 1110 = 6 || 8 + 6 * 15
7290 -> 408 = 6 || 8 * 6
7290 -> 40815 = 6 || 8 * 6 || 15
7290 -> 423 = 6 || 8 * 6 + 15
7290 -> 6120 = 6 || 8 * 6 * 15
7290 -> 14 = 6 + 8
7290 -> 146 = 6 + 8 || 6
7290 -> 14615 = 6 + 8 || 6 || 15
7290 -> 161 = 6 + 8 || 6 + 15
7290 -> 2190 = 6 + 8 || 6 * 15
7290 -> 20 = 6 + 8 + 6
7290 -> 2015 = 6 + 8 + 6 || 15
7290 -> 35 = 6 + 8 + 6 + 15
7290 -> 300 = 6 + 8 + 6 * 15
7290 -> 84 = 6 + 8 * 6
729

11387

In [23]:
answer_b = 0
for tv in tqdm(tvs):
    if tv.valid_b:
        answer_b += tv.total
answer_b

  0%|          | 0/850 [00:00<?, ?it/s]

271691107779347

In [24]:
puzzle.answer_b = answer_b

# Day 8

In [ ]:
puzzle = Puzzle(2022, 8)

In [ ]:
data = np.array([[int(number) for number in line] for line in puzzle.input_data.split("\n")])

In [ ]:
len_x, len_y = data.shape

## Part 1 

In [ ]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x - 1):
    for y in range(1, len_y - 1):
        number = data[x, y]
        if (
            np.all(data[:x, y] < number)
            or np.all(data[x+1:, y] < number)
            or np.all(data[x, :y] < number)
            or np.all(data[x, y+1:] < number)
         ):
            total_tree[x, y] = 1

In [ ]:
answer_a = np.sum(total_tree)
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x-1):
    for y in range(1, len_y-1):
        number = data[x, y]
        
        a = 0
        for new_n in data[x-1::-1, y]:
            a += 1
            if new_n >= number:
                break
                
        b = 0
        for new_n in data[x+1:, y]:
            b += 1
            if new_n >= number:
                break
        
        c = 0
        for new_n in data[x, y-1::-1]:
            c += 1
            if new_n >= number:
                break
                
        d = 0
        for new_n in data[x, y+1:]:
            d += 1
            if new_n >= number:
                break
                
        total_tree[x, y] = a * b * c * d       

In [ ]:
answer_b = np.max(total_tree)
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 9

In [ ]:
puzzle = Puzzle(2022, 9)

In [ ]:
data = puzzle.input_data

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def as_tuple(self):
        return self.x, self.y
    
    @classmethod
    def from_tuple(cls, coord):
        return Point(*coord)
                   
    @classmethod
    def create_vector(cls, direction):
        match direction:
            case "U":
                return cls(1, 0)
            case "R":
                return cls(0, 1)
            case "D":
                return cls(-1, 0)
            case "L":
                return cls(0, -1)
            case "_":
                raise ValueError("unkown direction")
        
    def one_tail_near(self, other_point):
        return abs(self.x - other_point.x) <= 1 and abs(self.y - other_point.y) <= 1
    
    def sign(self):
        return Point(np.sign(self.x), np.sign(self.y))
    
    def __sub__(self, other_point):
        return Point(self.x - other_point.x, self.y - other_point.y)
    
    def __isub__(self, other_point):
        return self - other_point
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __add__(self, other_point):
        return Point(self.x + other_point.x, self.y + other_point.y)
    
    def __iadd__(self, other_point):
        return self + other_point
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Point{self}"

In [ ]:
class RopeGame:
    def __init__(self, data, tail_lenght=1, start=(0,0)):
        self.instructions = [line.split(" ") for line in data.split("\n")]
        self.knots = [Point.from_tuple(start) for _ in range(tail_lenght + 1)]
        self.knots_positions = [set([start]) for _ in range(tail_lenght + 1)]
    
    def parse_instruction(self, instruction):
        direction, value = instruction
        vector = Point.create_vector(direction)
        
        for _ in range(int(value)):
            self.update_knot(0, self.knots[0] + vector)
            
            for idx, knot in list(enumerate(self.knots))[1:]:
                prev_knot = self.knots[idx-1]
                if knot.one_tail_near(prev_knot):
                    break
                
                deplacement = (prev_knot - knot).sign()
                self.update_knot(idx, knot + deplacement)
                
    def run(self):
        for instruction in self.instructions:
            self.parse_instruction(instruction)
            
    def update_knot(self, pos, new_knot):
        self.knots[pos] = new_knot
        self.knots_positions[pos].add(new_knot.as_tuple)
    
    @property
    def answer_a(self):
        return len(self.knots_positions[1])
    
    @property
    def answer_b(self):
        return len(self.knots_positions[-1])
    

## Part 1 

In [ ]:
rg = RopeGame(data)
rg.run()
rg.answer_a

In [ ]:
puzzle.answer_a = rg.answer_a

## Part 2

In [ ]:
rg = RopeGame(data, 9)
rg.run()

In [ ]:
rg.answer_a

In [ ]:
rg.answer_b

In [ ]:
puzzle.answer_b = rg.answer_b

# Day 10

In [ ]:
puzzle = Puzzle(2022, 10)

In [ ]:
data = puzzle.input_data.split("\n")

In [ ]:
def test_cycle(x, cycle):
    if cycle in {20, 60, 100, 140, 180, 220}:
        print(cycle, x)
        return cycle * x
    return 0

## Part 1 

In [ ]:
x = 1
cycle = 0
answer_a = 0

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_a += test_cycle(x, cycle)
        case "addx", value:
            cycle += 1
            answer_a += test_cycle(x, cycle)
            cycle += 1
            answer_a += test_cycle(x, cycle)
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
def draw(x, cycle):
    if x <= (cycle % 40) <= x+2:
        return "#"
    return "."

In [ ]:
def print_map(answer_b):
    for chunk in more_itertools.batched(answer_b, 40):
        print("".join(chunk))

In [ ]:
x = 1
cycle = 0
answer_b = []

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_b.append(draw(x, cycle))
        case "addx", value:
            cycle += 1
            answer_b.append(draw(x, cycle))
            cycle += 1
            answer_b.append(draw(x, cycle))
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")


In [ ]:
print_map(answer_b)

In [ ]:
puzzle.answer_b = "RJERPEFC"

# Day 11

In [ ]:
puzzle = Puzzle(2022, 11)

In [ ]:
data = puzzle.input_data

In [ ]:
data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [ ]:
class Monkey:
    def __init__(self, name,items, operation, test_val, monkey_true, monkey_false):
        self.item_inspected = 0
        self.name = name,
        self.items = items
        self.operation = operation
        self.test_val = test_val
        self.monkey_true = monkey_true
        self.monkey_false = monkey_false
    
    def inspect(self, old, p1=True):
        self.item_inspected += 1
        item = eval(self.operation)
        if p1:
            item = item // 3
        return item
        
    def test(self, item):
        monkey_dest = self.monkey_true if item % self.test_val == 0 else self.monkey_false
        return monkey_dest, item
    
    def add_item(self, item):
        self.items.append(item)
    
    def do_round(self, p1=True):
        for item in self.items:
            item = self.inspect(item, p1=p1)
            yield self.test(item)
        self.items = []

In [ ]:
class MonkeyManager:
    def __init__(self, data):
        # Create monkey
        self.monkeys = []
        self.test_total = 1
        self.data = data
        
        self.create()
    
    def create(self):

        for monkey_data in self.data.split("\n\n"):
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name = name.strip()
            items = list(map(int, number_pattern.findall(items)))
            operation = operation.split("= ")[1]
            test = int(number_pattern.search(test).group())
            self.test_total *= test
            m_true = int(number_pattern.search(m_true).group())
            m_false = int(number_pattern.search(m_false).group())
            new_monkey = Monkey(name, items, operation, test, m_true, m_false)
            self.monkeys.append(new_monkey)
            
    def do_round(self, p1=True):
        
        for monkey in self.monkeys:
            for monkey_dest, item in monkey.do_round(p1):
                self.monkeys[monkey_dest].add_item(item % self.test_total)
                
    def answer_a(self):
        self.reset()
        for i in range(20):
            self.do_round(p1=True)
        
        return self.monkey_inspections(2)
    
    
    def answer_b(self):
        self.reset()
        for i in tqdm.tqdm(range(10000)):
            self.do_round(p1=False)
        
        return self.monkey_inspections(2)
    
    
    def monkey_inspections(self, n):
        m_i = [monkey.item_inspected for monkey in self.monkeys]
        m_i = heapq.nlargest(2, m_i)
        return functools.reduce(operator.mul, m_i, 1)
        
    def reset(self):
        self.monkeys = []
        self.test_total = 1       
        self.create()

In [ ]:
mm = MonkeyManager(data)

## Part 1 

In [ ]:
answer_a = mm.answer_a()
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
answer_b = mm.answer_b()
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 12

In [ ]:
puzzle = Puzzle(2022, 12)

In [ ]:
data = puzzle.input_data

In [ ]:
data = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""

In [ ]:
def adjs(position, nodes):
    x, y = position
    adjs = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
    return [adj for adj in adjs if adj in nodes]

In [ ]:
def distance(pos1, pos2, nodes_weight):
    return nodes_weight[pos2] - nodes_weight[pos1]

In [ ]:
base_nodes = np.array([[letter for letter in line] for line in data.split("\n")])
nodes_weight = np.array([[ord(letter) - ord("a") for letter in line] for line in data.split("\n")])

In [ ]:
start = np.where(nodes_weight == ord("S") - ord("a"))
start = next(zip(start[0], start[1]))
nodes_weight[start] = 0

In [ ]:
end = np.where(nodes_weight == ord("E") - ord("a"))
end = next(zip(end[0], end[1]))
nodes_weight[end] = ord("z") - ord("a")

In [ ]:
nodes = list(np.ndindex(*nodes_weight.shape))

In [ ]:
distances = {
    node: {adj: 1 for adj in adjs(node, nodes) if distance(node, adj, nodes_weight) <= 1} for node in nodes
}

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_nodes_from(nodes)

In [ ]:
for node, adjs in distances.items():
    for adj, dist in adjs.items():
        G.add_edge(node, adj, weight=dist)

## Part 1 

In [ ]:
answer_a = nx.shortest_path_length(G, start, end)
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
starts = np.where(nodes_weight == 0)
starts

In [ ]:
dists = []
for start in zip(starts[0], starts[1]):
    try:
        dist = nx.shortest_path_length(G, start, end)
        dists.append(dist)
    except nx.NetworkXNoPath:
        continue
        
answer_b = min(dists)
answer_b


In [ ]:
puzzle.answer_b = answer_b

# Day 13

In [ ]:
puzzle = Puzzle(2022, 13)

In [ ]:
signals_pairs = [tuple(map(eval, line.split("\n"))) for line in puzzle.input_data.split("\n\n")]

## Part 1 

In [ ]:
def parse_pair(p1, p2):
    """ -1 --> a > b
    0 --> a == b
    1 --> a < b
    
    """
    if isinstance(p1, int) and isinstance(p2, int):
        return np.sign(p2 - p1)

    for e1, e2 in zip(p1, p2):
        match e1, e2:
            case int(), list():
                e1 = [e1]
            case list(), int():
                e2 = [e2]
            
        if (res := parse_pair(e1, e2)) != 0:
            return res
    
    return np.sign(len(p2) - len(p1))

In [ ]:
res = sum(idx + 1 for idx, pair in enumerate(signals_pairs) if parse_pair(*pair) != -1)

In [ ]:
res

In [ ]:
puzzle.answer_a = res

## Part 2

In [ ]:
signals = [eval(line) for line in puzzle.input_data.split("\n") if line]

In [ ]:
s1 = [[2]]
signals.append(s1)
s2 = [[6]]
signals.append(s2)

In [ ]:
def compare(l1, l2):
    res = parse_pair(l1, l2)
    return -1 if res != -1 else 1

In [ ]:
signals = sorted(signals, key=functools.cmp_to_key(compare))

In [ ]:
filter_signals = [idx + 1 for idx, signal in enumerate(signals) if signal == s1 or signal == s2]
tot = functools.reduce(operator.mul, filter_signals, 1)

In [ ]:
tot

In [ ]:
puzzle.answer_b = tot

# Day 14

In [ ]:
puzzle = Puzzle(2022, 14)

In [ ]:
data = puzzle.input_data

In [ ]:
data_test = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9"""

In [ ]:
sand_entry_x, send_entry_y = sand_entry = (500, 0)

In [ ]:
def get_cave_extremity(cave, p1=True):
    y_min = send_entry_y
    y_max = max(map(lambda t: t[1], cave)) + (0 if p1 else 2)
    x_min = min(map(lambda t: t[0], cave))
    x_max = max(map(lambda t: t[0], cave))
    
    return (x_min, x_max), (y_min, y_max)

In [ ]:
def init_cave(data, p1=True):
    coords = [[tuple(map(int, coord.split(","))) for coord in line.split(" -> ")] for line in data.split("\n")]
    cave = collections.defaultdict(str)

    def get_numbers_inside(n1, n2):
        return range(min(n1, n2), max(n1, n2) + 1)

    for coord in coords:

        for (x1, y1), (x2, y2) in itertools.pairwise(coord):
            if x1 == x2:
                for yi in get_numbers_inside(y1, y2):
                    cave[(x1, yi)] = "#"
            # y1 == y2
            else:
                for xi in get_numbers_inside(x1, x2):
                    cave[(xi, y1)] = "#"
                    
    if not p1:
        (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
        boottom_half_lenght = y_max - y_min
        
        for xi in range(- boottom_half_lenght, boottom_half_lenght + 1):
            cave[(xi + sand_entry_x, y_max)] = "#"
        
    return cave



In [ ]:
np.set_printoptions(linewidth=150)
def print_cave(cave, p1=True):
    cave = dict(cave)
    (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
    representation = np.chararray((x_max - x_min + 1, y_max - y_min + 1), unicode=True)
    representation[:, :] = "."
    for (x, y), value in cave.items():
        representation[x - x_min, y] = value
    representation[sand_entry_x - x_min, y_min] = "x"
    print(representation.T)

In [ ]:
cave_test = init_cave(data_test, p1=False)
print_cave(cave_test)

## Part 1

In [ ]:
def compute(data, p1=True):
    cave = init_cave(data, p1=p1)
    time = -1
    (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
    while True:
        time += 1
        sand_pos_x, sand_pos_y = sand_entry
        
        if cave[sand_entry] == "o":
            return cave, time - 1
        
        while True:
            under = (sand_pos_x, sand_pos_y + 1)

            match cave[under]:
                case "#" | "o":
                    under_left  = (sand_pos_x - 1, sand_pos_y + 1)
                    under_right = (sand_pos_x + 1, sand_pos_y + 1)

                    if cave[under_left] == "":
                        sand_pos_x, sand_pos_y = under_left
                    elif cave[under_right] == "":
                        sand_pos_x, sand_pos_y = under_right
                    else:
                        cave[(sand_pos_x, sand_pos_y)] = "o"
                        break
                case "":
                    sand_pos_x, sand_pos_y = under

            if sand_pos_y == y_max:
                return cave, time

In [ ]:
cave, time = compute(data_test)
print(f"{time=}")
print_cave(cave)

In [ ]:
_, answer_a = compute(data)
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
cave, time = compute(data_test, p1=False)
print(f"{time=}")
print_cave(cave, p1=False)

In [ ]:
_, answer_b = compute(data, p1=False)
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 15

In [ ]:
puzzle = Puzzle(2022, 15)

In [ ]:
data = puzzle.input_data

In [ ]:
data_test = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def as_tuple(self):
        return self.x, self.y
    
    @classmethod
    def from_tuple(cls, coord):
        return Point(*coord)
      
    def manathan_distance(self, other_point):
        return abs(self.x - other_point.x) + abs(self.y - other_point.y)
    
    def det(self, point):
        return self.x * point.y - self.y * point.x
    
    def __sub__(self, other_point):
        return Point(self.x - other_point.x, self.y - other_point.y)
    
    def __isub__(self, other_point):
        return self - other_point
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __iadd__(self, other_point):
        return self + other_point
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Point{self}"
    
class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
    def det(self):
        return self.p1.det(self.p2)
    
    def intersection(self, line):
        xdiff = Point(self.p1.x - self.p2.x, line.p1.x - line.p2.x)
        ydiff = Point(self.p1.y - self.p2.y, line.p1.y - line.p2.y)
        
        div = xdiff.det(ydiff)
        if div == 0:
            return False
        
        d = Point(self.det(), line.det())
        x = d.det(xdiff) // div
        y = d.det(ydiff) // div
        
        return Point(x, y)

class Sensor:
    def __init__(self, sensor, beacon):
        self.sensor = sensor
        self.beacon = beacon
        self.distance = sensor.manathan_distance(beacon)
        self.outer_distance = self.distance + 1
        self.lines = self.possible_outer_lines()

        
    def possible_outer_points(self):
        x, y = self.sensor.as_tuple
        return [
            Point(x - self.outer_distance, y),
            Point(x, y - self.outer_distance),
            Point(x + self.outer_distance, y),
            Point(x, y + self.outer_distance)
        ]
    
    def possible_outer_lines(self):
        points_i = self.possible_outer_points()
        points_j = points_i[1:] + [points_i[0]]
        
        return [Line(pi, pj) for (pi, pj) in zip(points_i, points_j)]
            
    def point_possible(self, point):
        return self.sensor.manathan_distance(point) > self.distance
    
    def outer_intersections_sensor(self, other_sensor, start, end):
        for line_i in self.lines:
            for line_j in other_sensor.lines:
                if (intersection := line_i.intersection(line_j)):
                    if start <= intersection.x <= end and start <= intersection.y <= end:
                        yield intersection       

In [ ]:
number_pattern = re.compile("-?\d+")
def parse_data(data):
    sensors_beacons = []
    for line in data.split("\n"):
        s_x, s_y, b_x, b_y = map(lambda match: int(match.group()), number_pattern.finditer(line))
        sensor, beacon = Point(s_x, s_y), Point(b_x, b_y)
        sensors_beacons.append((sensor, beacon, sensor.manathan_distance(beacon)))
        
    return sensors_beacons

In [ ]:
number_pattern = re.compile("-?\d+")
def parse_data2(data):
    sensors = []
    for line in data.split("\n"):
        s_x, s_y, b_x, b_y = map(lambda match: int(match.group()), number_pattern.finditer(line))
        sensor, beacon = Point(s_x, s_y), Point(b_x, b_y)
        sensors.append(Sensor(sensor, beacon))
    return sensors

## Part 1

In [ ]:
def span(sensor, beacon, m_s_b, y):
    """
    find x
    sensor.manathan(x, y) = sensor.manathan(beacon)
    abs(sx - x) + abs(sy - y) = sensor.manathan(beacon)
    abs(sx - x) = sensor.manathan(beacon) - abs(sy - y) = z avec z < 0
    sx - x = z AND sx - x = -z
    x = sx -z and x = sx + z
    """
    z = m_s_b - abs(sensor.y - y)
    if z < 0:
        return range(0)
    
    x1, x2 = sensor.x - z, sensor.x + z 
    
    return range(min(x1, x2), max(x1, x2) + 1)

def span_2(sensor, beacon, m_s_b, y):
    """
    find x
    sensor.manathan(x, y) = sensor.manathan(beacon)
    abs(sx - x) + abs(sy - y) = sensor.manathan(beacon)
    abs(sx - x) = sensor.manathan(beacon) - abs(sy - y) = z avec z < 0
    sx - x = z AND sx - x = -z
    x = sx -z and x = sx + z
    """
    z = m_s_b - abs(sensor.y - y)
    if z < 0:
        return False
    
    x1, x2 = sensor.x - z, sensor.x + z 
    
    return [min(x1, x2), max(x1, x2)]

In [ ]:
def part_1_2(data, y):
    line_y = []
    b_s = set()
    for sensor, beacon, m_s_b in parse_data(data):
        if sensor.y == y:
            b_s.add(sensor.x)
        if beacon.y == y:
            b_s.add(beacon.x)
        if (r := span_2(sensor, beacon, m_s_b, y)):
            line_y.append(r)
            
    
    line_y.sort(key=lambda interval: interval[0])
    merged = [line_y[0]]
    for current in line_y:
        previous = merged[-1]
        if current[0] <= previous[1]:
            previous[1] = max(previous[1], current[1])
        else:
            merged.append(current)
            
    return sum(r_e - r_s + 1 for r_s, r_e in merged) - sum(r_s <= s <= r_e for r_s, r_e in merged for s in b_s)


In [ ]:
def part_1_1(data, y):
    line_y = set()
    b_s = set()
    for sensor, beacon, m_s_b in parse_data(data):
        if sensor.y == y:
            b_s.add(sensor.x)
        if beacon.y == y:
            b_s.add(beacon.x)
        line_y |= set(span(sensor, beacon, m_s_b, y))
    line_y -= b_s
    return len(line_y)

In [ ]:
part_1_2(data_test, 10)

In [ ]:
part_1_1(data_test, 10)

In [ ]:
part_1_1(data, 2000000) == part_1_2(data, 2000000)

In [ ]:
%%timeit
part_1_1(data, 2000000)

In [ ]:
%%timeit
part_1_2(data, 2000000)

In [ ]:
answer_a = part_1_2(data, 2000000)
puzzle.answer_a = answer_a

## Part 2

In [ ]:
def find_outer_points(sensor, m_s_b):
    outer_distance = m_s_b + 1
    sx, sy = sensor.as_tuple
    outers = []
    for i in range(outer_distance):
        yield Point(sx - outer_distance + i, sy + i)
        yield Point(sx - outer_distance + i, sy - i)
        yield Point(sx + i, sy + outer_distance - i)
        yield Point(sx - i, sy + outer_distance - i)

In [ ]:
def all_possible_point(sensors_beacons, start, end):
    for sensor, _, m_s_b in sensors_beacons:
        for point in find_outer_points(sensor, m_s_b):
            if start <= point.x <= end and start <= point.y <= end:
                yield point
                
def compute(data, start=0, end=20):
    sensors_beacons = parse_data(data)
    
    for point in tqdm.tqdm(all_possible_point(sensors_beacons, start, end)):
        for sensor, _, m_s_b in sensors_beacons:
            if sensor.manathan_distance(point) <= m_s_b:
                break
        else:
            print(point)
            return point

In [ ]:
compute(data_test, 0, 20)

In [ ]:
res = compute(data, 0, 4000000)

In [ ]:
answer_b = res.x*4000000 + res.y
answer_b

In [ ]:
puzzle.answer_b = answer_b

In [ ]:
def all_possible_point_with_line(sensors, start, end):
    for si, sj in itertools.combinations(sensors, 2):
        yield from si.outer_intersections_sensor(sj, start, end)

In [ ]:
def compute2(data, start=0, end=4000000):
    sensors = parse_data2(data)
    for point in all_possible_point_with_line(sensors, start, end):
        for sensor in sensors:
            if not sensor.point_possible(point):
                break
        else:
            return point

In [ ]:
compute2(data_test, 0, 20)

In [ ]:
%%timeit
res2 = compute2(data, 0, 4000000)

In [ ]:
res2 = compute2(data, 0, 4000000)
answer_b_2 = res2.x*4000000 + res2.y

In [ ]:
answer_b_2 == answer_b